# Cours ENPC - Pratique du calcul scientifique

## Interpolation et approximation

Les graphiques peuvent être tracés grâce à la bibliothèque [`Plots.jl`](https://github.com/JuliaPlots/Plots.jl)

- [dépôt GitHub](https://github.com/JuliaPlots/Plots.jl)

- [documentation](https://docs.juliaplots.org/stable/)

- [tutoriel](https://docs.juliaplots.org/stable/tutorial/)

- Il est possible de définir des options de tracé par défaut (qui peuvent être redéfinies ponctuellement au besoin) par

   ```julia
   Plots.default(fontfamily="Computer Modern",
               titlefontsize=20,
               xlabelfontsize=20,
               ylabelfontsize=20,
               legendfontsize=16,
               xtickfontsize=16,
               ytickfontsize=16,
               linewidth=2,
               markersize=7,
               framestyle=:box,
               label=nothing,
               grid=true)
   ```

In [ ]:
# Quelques bibliothèques utiles. Vous pouvez modifier les paramètres à votre guise et ajouter des bibliothèques
# si besoin (sauf s'il explicitement mentionné dans un exercice de ne pas utiliser de bibliothèque externe)

using Polynomials, LinearAlgebra, Plots, LaTeXStrings

Plots.default(fontfamily="Computer Modern",
              titlefontsize=20,
              xlabelfontsize=20,
              ylabelfontsize=20,
              legendfontsize=12,
              xtickfontsize=12,
              ytickfontsize=12,
              framestyle=:box,
              label=nothing,
              grid=true)

# For information on any of these, type
# plotattr("framestyle")

### Exercice sur l'interpolation de Lagrange en arthmétique inexacte

On étudie la performance de l’interpolation de Lagrange en arithmétique inexacte.

1.  Créer une fonction `get_lagrange_interpolant` en `Julia` qui accepte comme arguments un vecteur `x` de nœuds équidistants et un vecteur `u` de valeurs d'une fonction $u$, que l'on veut interpoler en ces nœuds, et qui construit et renvoie l'interpolant de Lagrange, $\hat{u}$ comme la fonction `interpolant` en `Julia`. Assurez-vous que dans l'intérieur de votre fonction l'arithmétique est faite dans la même précision que l'arguement qu'elle reçoit. Indice: Regardez la documentation des fonctions `one` et `zero`.

1.  Tester votre fonction `get_lagrange_interpolant` sur la fonction $f:[-1, 1] \to \mathbb{R}$, $f(x) = 1 $ pour tout $x \in [-1, 1]$ pour plusieurs $n$ et avec des nœuds et valeurs de type `Float64`, `Float32`, et `Float16`.

    Commenter ce qui se passe lorsque l'on utilise plus de nœuds et les types de moindre précision. Que peut expliquer ce comportement ? Quel est l'interpolant à $n$ nœuds équidistants de $f$ en arithmétique exacte ?

1. *Facultatif :* examiner le cas d'une précision à 128 bits (type `Float128` disponible avec la bibliothèque [`Quadmath.jl`](https://github.com/JuliaMath/Quadmath.jl)

In [ ]:
function get_lagrange_interpolant(x, u)
    """
    Définir une fonction qui renvoie une fonction `interpolant` qui 
    est une interpolation de Lagrange de noeuds `x` et valeurs `u`
    """
    function interpolant(y)
        """
        Interpolant û comme il est défini dans le poly
        """
        # Ajouter votre code ici : la fonction doit retourner la valeur de û au point y
        s = zero(eltype(u))

        n = length(x)
        for i=1:n
            s += prod((y-x[j])/(x[i]-x[j]) for j=1:n if j!=i)
        end

        return s
    end
    
    return interpolant
end

In [ ]:
# Pour tester votre fonction, vous pouvez utiliser la fonction suivante en tournant les cellules qui suivent

using Plots
function lagrange_tester(n, m, num_type, fun)
    """
    Tester `get_lagrange_interpolant` sur la fonction `fun`
    pour tout x avec des noeud d'une précision donné par `num_type`.
    Appliquer l'interpolant renvoyé par `get_lagrange_interpolant`
    à `m` point équidistant
    
    Arguments
    ---------
    n : nombre des noeuds
    
    m : nombre des points à évaluer l'interpolant
    
    num_type : type de noeuds et valeurs d'interpolation, ex. Float64
    
    fun: fonction à interpoler
        
    Retour
    ------
    
    p1 : plot de l'interpolation evalué à n+1 points equidistants et la vraie fonction
    """
    # créer n noeuds equidistants et n valeurs de type num_type
    x = LinRange{num_type}(-1, 1, n)
    # idem avec m noeuds pour le tracé
    x_plot = LinRange{num_type}(-1, 1, m)
    û = get_lagrange_interpolant(x, fun.(x))
    
    # Dessiner l'interpolation avec m points equidistant
    x_plot = LinRange{num_type}(-1, 1, m)
    p1 = plot(x_plot, û.(x_plot), label = "Interpolation")
    
    # Dessiner la vraie fonction 
    plot!(x_plot, fun.(x_plot), label = "Vraie Fonction")
    
    print("Temps d'evaluation d'interpolant: ")
    @time û.(x_plot)
    
    print("Erreur d'interpolation: $(maximum(x -> abs(û(x)-fun(x)), x_plot))") 
        
    return p1
end

In [ ]:
m = 200
fun(x) = 1

In [ ]:
lagrange_tester(31, m, Float64, fun)

In [ ]:
lagrange_tester(41, m, Float64, fun)

In [ ]:
lagrange_tester(51, m, Float64, fun)

In [ ]:
lagrange_tester(61, m, Float64, fun)

In [ ]:
lagrange_tester(11, m, Float32, fun)

In [ ]:
lagrange_tester(21, m, Float32, fun)

In [ ]:
lagrange_tester(11, m, Float16, fun)

In [ ]:
lagrange_tester(21, m, Float16, fun)

In [ ]:
lagrange_tester(101, m, Float64, fun)

In [ ]:
using Quadmath

lagrange_tester(101, m, Float128, fun)

In [ ]:
lagrange_tester(121, m, Float128, fun)

In [ ]:
lagrange_tester(131, m, Float128, fun)

In [ ]:
lagrange_tester(141, mj, Float128, fun)

### Exercice sur les noeuds d'interpolation

Écrire un code `Julia` pour interpoler la fonction suivante à l'aide d'un polynôme de degré 20 sur l'intervalle $[-1, 1]$.
$$
    f(x) = \tanh\left(\frac{x+1/2}{\varepsilon}\right) + \tanh\left(\frac{x}{\varepsilon}\right) + \tanh\left(\frac{x-1/2}{\varepsilon}\right),
    \qquad \varepsilon = .01
$$
Utiliser des noeuds équidistants puis des noeuds de Tchebychev et comparer les deux approches en termes de précision.
Tracer la fonction $f$ ainsi que les polynômes d'interpolation.

*Indications :*

- Pour limiter les erreurs d'arrondi numérique, il est préférable que la fonction renvoie un type `BigFloat`, autrement dit

    ```julia
    f(x) = BigFloat(tanh((x+1/2)/ε) + tanh(x/ε) + tanh((x-1/2)/ε))
    ```

- Pour calculer rapidement les noeuds de Tchebychev, on peut exploiter la macro `@.` (comme toujours, il est conseillé de se référer à la documentation d'une commande en tapant `?` puis la commande dans la console). Cette commande évite d'utiliser des `.` après chaque fonction ou avant chaque opérateur.

    ```julia
    x = @. -cos(π*((0:n-1)+1/2)/n)
    ```

- Le calcul du polynôme interpolateur peut être obtenu par la fonction [`fit`](https://juliamath.github.io/Polynomials.jl/stable/#Fitting-arbitrary-data) de la bibliothèque [`Polynomials.jl`](https://github.com/JuliaMath/Polynomials.jl).

- Il peut être utile pour comparer les deux interpolations de limiter les valeurs minimale et maximale sur l'axe `y` à l'aide de l'option `ylims = (ymin,ymax)` dans une fonction de tracé `plot`, `scatter`, ou leurs équivalents terminant par `!`. On rappelle que, par convention en `Julia` (et non par obligation), une fonction dont le nom se termine par `!` modifie ses arguments. Dans le cas d'un graphe, la première commande initiant le graphe ne doit pas comporter de `!` (`plot`, `scatter`, ...) tandis que les suivantes incrémentant le même graphe se terminent par `!` (`plot!`, `scatter!`, ...). Toute omission du `!` est considéré comme un *redémarrage à zéro* du tracé.

- Pour calculer la norme infinie d'une fonction afin d'évaluer la précision de l'interpolation, on pourra exploiter la fonction [`norm(...,Inf)`](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.norm) de la bibliothèque `LinearAlgebra` avec une échantillonnage suffisamment fin des valeurs de la fonction ou exploiter la fonction `maximum`. Noter que la conversion d'un nombre `y` de type `BigFloat` en `Float64` se fait par `convert(Float64,y)` ou plus simplement ici `Float64(y)`.


### Exercice sur les noeuds d'interpolation : animation de graphes

En exploitant la macro [`@animate`](https://docs.juliaplots.org/latest/animations/) de la bibliothèque `Plots.jl`, créer une animation permettant de voir l'évolution superposée des interpolations avec noeuds équidistants et de Tchebychev de la fonction de Runge $u(x)=\frac{1}{1+25x^2}$ sur l'intervalle $[-1,1]$. On fera varier le nombre de noeuds par exemple de 2 à 50 et on pourra tenir compte des indications de l'exercice précédent pour limiter par exemple l'extension en `y`.

### Exercice sur l'équation de Laplace

On se propose d'implémenter une méthode numérique pour résoudre approximativement l'équation de Laplace avec conditions au bord homogène de Dirichlet:

$$ u\in C^2([0,1]),\quad\left\{\begin{aligned}  -u''(x) & = f(x) & \forall\, x\in(0,1),\\ u(0) & = u(1) = 0. \end{aligned}\right.$$
Pour cela, on approxime $f$ avec un polynome interpolateur $\hat f$, puis on résoud exactement l'équation de Laplace associée. Implémenter cette méthode.
On pourra tester le cas où
$$f(x) = \exp(\sin(2\pi x))(\sin(2\pi x)-\cos(2\pi x)^2),$$
auquel cas la solution analytique est donnée par
$$ u(x)=(2\pi)^{-2}\exp((\sin(2\pi x))-1).$$

*Indications :*
- On pourra utiliser la fonction `fit` de la bibliothèque `Polynomials.jl` pour obtenir le polynôme interpolateur:
    ```julia
     p = fit(x,y)
    ```
où `x` sont les noeuds d'interpolation et `y` sont les valeurs de la fonction à interpoler.
- Pour calculer la solution analytique avec membre de droite polynomial, on pourra remarquer que toutes les solutions sont des polynômes, et que, sans condition au bord, la solution est unique modulo $\mathbf{P}_1$.
- On pourra utiliser la fonction `integrate` de la bibliothèque `Polynomials.jl` qui calcule une primitive d'un polynôme:
    ```julia
    P = integrate(p)
    ```
- Utiliser le format `BigFloat` pour limiter les erreurs d'arrondi